In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
plt.rcParams["figure.figsize"] = (12, 10)

In [ ]:
fits_file = "supernovae/tess-s0001-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sn = tess_cpm.Source(fits_file, remove_bad=True)

In [ ]:
sn.set_aperture(rowrange=[49, 52], colrange=[49, 52])
sn.plot_cutout(rowrange=[45, 55], colrange=[45, 55], show_aperture=True)

In [ ]:
sn.plot_pix_by_pix(data_type="normalized_flux")

In [ ]:
sn.add_cpm_model()
# sn.set_regs([0.01])
# sn.holdout_fit_predict(k=2);

In [ ]:
# sn.plot_pix_by_pix(data_type="rescaled_cpm_subtracted_lc", split=True)

In [ ]:
# aperture_lc = sn.get_aperture_lc(data_type="cpm_subtracted_lc")
# cpm_prediction = sn.get_aperture_lc(data_type="cpm_prediction")

In [ ]:
# plt.plot(sn.time, aperture_lc)
# plt.plot(sn.time, cpm_prediction)

In [ ]:
sn.add_poly_model()
sn.set_regs([0.001, 0.1])
sn.holdout_fit_predict(k=50);

In [ ]:
sn.plot_pix_by_pix(data_type="rescaled_cpm_subtracted_lc", split=True)
# sn.plot_pix_by_pix(data_type="cpm_prediction")
# sn.plot_pix_by_pix(data_type="poly_model_prediction")

In [ ]:
flux = sn.get_aperture_lc(data_type="normalized_flux")
cpm_prediction = sn.get_aperture_lc(data_type="cpm_prediction")
# poly_prediction = sn.get_aperture_lc(data_type="poly_model_prediction")
apt_lc = sn.get_aperture_lc(data_type="rescaled_cpm_subtracted_lc")

In [ ]:
# plt.plot(sn.time, flux)
# plt.plot(sn.time, cpm_prediction)
# plt.plot(sn.time, poly_prediction)
plt.plot(sn.time, apt_lc)

In [ ]:
fits_file = "supernovae/tess-s0002-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sn2 = tess_cpm.Source(fits_file, remove_bad=True)

In [ ]:
sn2.set_aperture(rowrange=[49, 52], colrange=[49, 52])
sn2.plot_cutout(rowrange=[45, 55], colrange=[45, 55], show_aperture=True)

In [ ]:
sn2.plot_pix_by_pix(data_type="raw")

In [ ]:
sn2.add_cpm_model()
sn2.add_poly_model()
sn2.set_regs([0.001, 0.1])
sn2.holdout_fit_predict(k=50);

In [ ]:
sn2.plot_pix_by_pix(data_type="rescaled_cpm_subtracted_lc", split=True)
sn2.plot_pix_by_pix(data_type="poly_model_prediction", split=True)

In [ ]:
lc = sn2.get_aperture_lc(data_type="rescaled_cpm_subtracted_lc")

In [ ]:
plt.plot(sn2.time, lc)

In [ ]:
plt.plot(sn.time, apt_lc)
plt.plot(sn2.time, lc + np.abs(apt_lc[-1] - lc[0]))